In [1]:
import spams
import pickle
from scipy.spatial import distance
from time import time
import numpy as np
import os
import imageio
import matplotlib.pyplot as plt
import scipy
import scipy.stats
from joblib import Parallel, delayed
import pandas as pd
%matplotlib inline

In [2]:
with open("../data/LFW_DATA.pickle", "rb") as f:
    lfw = pickle.load(f)
with open("../data/lfw_attributes.txt", "r") as f:
    lfw_attributes = f.readlines()

In [3]:
attributes = lfw_attributes[1].strip().split("\t")[1:]
print(len(attributes))
row = lfw_attributes[2].strip().split("\t")
print(len(row))

column_names = attributes
attr_feature = []
for i in lfw_attributes[2:]:
    row = i.strip().split("\t")
    attr_feature.append(row)
attr_feature = np.array(attr_feature)

75
75


In [4]:
attr_df = pd.DataFrame(attr_feature)
attr_df.columns = column_names

def replace(string):
    return string.replace(" ","_")
def zero_pad(string):
    return string.zfill(4)
attri_identity = attr_df["person"].apply(replace) + "_"+ attr_df["imagenum"].apply(zero_pad)

male_dict = {}
for i in range(len(attr_df)):
    male_dict[attri_identity[i]] = attr_df["Male"][i]

In [18]:
male_dict["Abbas_Kiarostami_0001"]

'0.871569840529'

In [56]:
database_id = [lfw["database_identity"][i][0][0].split("\\")[1][:-4] for i in range(len(lfw["database_identity"]))]
query_id = [lfw["query_identity"][i][0][0].split("\\")[1][:-4] for i in range(len(lfw["query_identity"]))]


database_attri = []
query_attri = []
for i, name in enumerate(database_id):
    try:
        database_attri.append(float(male_dict[name]))
    except:
        print(name)
        database_attri.append(0.)
print("query stage\n")
for i, name in enumerate(query_id):
    try:
        query_attri.append(float(male_dict[name]))
    except:
        print(name)
        query_attri.append(0.)

# initial a vector
K = 100
sigma = 120
a = np.array([-1 for _ in range(K//2)] + [1 for _ in range(K//2)])
Z_database = []
Z_query = []
for attri in database_attri:
    z = np.exp(abs(a-attri)/sigma)
    Z_database.append(z)
Z_database = np.array(Z_database)

for attri in query_attri:
    z = np.exp(abs(a-attri)/sigma)
    Z_query.append(z)
Z_query = np.array(Z_query)

In [68]:
np.save("Z_database.npy", Z_database)
np.save("Z_query.npy", Z_query)